In [1]:
import time, os, sys
import pandas as pd
import numpy as np
# from utilfunc import parabolic, freq_from_fft
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
import copy
import random
import datetime
from dateutil.parser import parse

C:\Anaconda3\envs\ml\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Anaconda3\envs\ml\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Anaconda3\envs\ml\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


In [2]:
# Path='./Raw_data/data.xlsx'
# Fault_number = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,
#                 164,254,326,327,392,420,456,457,458,459,460,461,462,537,538,546,
#                 552,561,607,608,611,632,799,780,878,920,921,943,944,945,959]
# dataset_dir = "./Dataset1"

Path='./Raw_data/data_2.xlsx'
Fault_number = [1,2,3,4,5,6,194,195,234,294,335,336,370,467,492]
dataset_dir = "./Dataset2"

if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)
else:
    print("The folder already exists.")

In [3]:
start_time = time.time()
df = pd.read_excel(Path)
print("--- %s seconds ---"%(time.time()-start_time))

--- 22.650049686431885 seconds ---


In [4]:
df.head()

,number,time,Temperature 1,Temperature 2,Temperature 3,Temperature 4,Temperature 5,Temperature 6,Temperature 7,Temperature 8,...,Temperature 22,Temperature 23,Temperature 24,Temperature 25,Unnamed: 27,Pressure 1,Pressure 2,Pressure 3,Pressure 4,Pressure 5
0,1.0,2019-11-6 12:32:52.500,130.75,142.75,94.25,149.75,89.50,111.25,116.25,203.50,...,101.00,686.75,666.75,322.00,NaN,0.0,1077.12,1406.71,1397.55,1082.61
1,NaN,2019-11-6 12:32:53.000,120.75,143.50,102.25,151.50,89.75,125.75,117.25,203.50,...,101.00,687.75,667.00,321.50,NaN,0.0,1110.08,1403.50,1397.09,1085.36
2,NaN,2019-11-6 12:32:53.500,145.50,152.25,126.25,163.25,87.75,116.50,130.50,203.50,...,100.25,657.75,667.00,280.25,NaN,0.0,1097.26,1409.00,1398.47,1101.38
3,NaN,2019-11-6 12:32:54.000,142.75,160.00,140.50,173.00,85.75,135.25,144.25,203.25,...,100.50,641.50,666.25,269.50,NaN,0.0,1096.34,1406.25,1397.55,1107.33
4,NaN,2019-11-6 12:32:54.500,162.00,166.50,152.00,179.25,83.25,126.00,155.50,203.25,...,100.00,642.75,666.75,268.50,NaN,0.0,1097.26,1407.62,1397.09,1098.63


In [5]:
df.index = pd.to_datetime(df.time)

In [6]:
df.head()

,number,time,Temperature 1,Temperature 2,Temperature 3,Temperature 4,Temperature 5,Temperature 6,Temperature 7,Temperature 8,...,Temperature 22,Temperature 23,Temperature 24,Temperature 25,Unnamed: 27,Pressure 1,Pressure 2,Pressure 3,Pressure 4,Pressure 5
time,,,,,,,,,,,,,,,,,,,,,
2019-11-06 12:32:52.500,1.0,2019-11-6 12:32:52.500,130.75,142.75,94.25,149.75,89.50,111.25,116.25,203.50,...,101.00,686.75,666.75,322.00,NaN,0.0,1077.12,1406.71,1397.55,1082.61
2019-11-06 12:32:53.000,NaN,2019-11-6 12:32:53.000,120.75,143.50,102.25,151.50,89.75,125.75,117.25,203.50,...,101.00,687.75,667.00,321.50,NaN,0.0,1110.08,1403.50,1397.09,1085.36
2019-11-06 12:32:53.500,NaN,2019-11-6 12:32:53.500,145.50,152.25,126.25,163.25,87.75,116.50,130.50,203.50,...,100.25,657.75,667.00,280.25,NaN,0.0,1097.26,1409.00,1398.47,1101.38
2019-11-06 12:32:54.000,NaN,2019-11-6 12:32:54.000,142.75,160.00,140.50,173.00,85.75,135.25,144.25,203.25,...,100.50,641.50,666.25,269.50,NaN,0.0,1096.34,1406.25,1397.55,1107.33
2019-11-06 12:32:54.500,NaN,2019-11-6 12:32:54.500,162.00,166.50,152.00,179.25,83.25,126.00,155.50,203.25,...,100.00,642.75,666.75,268.50,NaN,0.0,1097.26,1407.62,1397.09,1098.63


In [7]:
T_columns = [f"Temperature {i}" for i in range(1, 8)]
P_columns = [f"Pressure {i}" for i in range(1, 4) ]
print(T_columns, P_columns)

['Temperature 1', 'Temperature 2', 'Temperature 3', 'Temperature 4', 'Temperature 5', 'Temperature 6', 'Temperature 7'] ['Pressure 1', 'Pressure 2', 'Pressure 3']


In [8]:
df.shape

(44410, 33)

In [9]:
Index = df.number.dropna()
print(Index.shape)
Index.head(10)

(503,)


time
2019-11-06 12:32:52.500     1.0
2019-11-06 12:33:39.500     2.0
2019-11-06 12:34:15.000     3.0
2019-11-06 12:34:50.000     4.0
2019-11-06 12:35:25.500     5.0
2019-11-06 12:36:01.000     6.0
2019-11-06 12:36:36.500     7.0
2019-11-06 12:37:12.000     8.0
2019-11-06 12:37:47.500     9.0
2019-11-06 12:38:23.000    10.0
Name: number, dtype: float64

In [10]:
T_window=10
for channel_name in T_columns:
	df[channel_name+'_trend'] = df[channel_name].rolling(window=T_window).mean()
	#df[channel_name+'_trend']=df[channel_name+'_trend'].rolling(window=T_window).mean()
	df[channel_name+'_detrend'] = df[channel_name] - df[channel_name+'_trend']	
	
P_window=10        
for channel_name in P_columns:
	df[channel_name+'_trend'] = df[channel_name].rolling(window=P_window).mean()
	#df[channel_name+'_trend']=df[channel_name+'_trend'].rolling(window=P_window).mean()
	df[channel_name+'_detrend'] = df[channel_name]-df[channel_name+'_trend']

In [11]:
df.head(10)

,number,time,Temperature 1,Temperature 2,Temperature 3,Temperature 4,Temperature 5,Temperature 6,Temperature 7,Temperature 8,...,Temperature 6_trend,Temperature 6_detrend,Temperature 7_trend,Temperature 7_detrend,Pressure 1_trend,Pressure 1_detrend,Pressure 2_trend,Pressure 2_detrend,Pressure 3_trend,Pressure 3_detrend
time,,,,,,,,,,,,,,,,,,,,,
2019-11-06 12:32:52.500,1.0,2019-11-6 12:32:52.500,130.75,142.75,94.25,149.75,89.50,111.25,116.25,203.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-06 12:32:53.000,NaN,2019-11-6 12:32:53.000,120.75,143.50,102.25,151.50,89.75,125.75,117.25,203.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-06 12:32:53.500,NaN,2019-11-6 12:32:53.500,145.50,152.25,126.25,163.25,87.75,116.50,130.50,203.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-06 12:32:54.000,NaN,2019-11-6 12:32:54.000,142.75,160.00,140.50,173.00,85.75,135.25,144.25,203.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-06 12:32:54.500,NaN,2019-11-6 12:32:54.500,162.00,166.50,152.00,179.25,83.25,126.00,155.50,203.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-06 12:32:55.000,NaN,2019-11-6 12:32:55.000,154.50,170.00,159.75,183.50,82.00,142.75,163.00,203.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-06 12:32:55.500,NaN,2019-11-6 12:32:55.500,170.25,173.00,163.75,186.50,81.00,131.50,169.25,203.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-06 12:32:56.000,NaN,2019-11-6 12:32:56.000,160.25,175.50,165.25,188.50,78.25,147.25,173.00,202.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-06 12:32:56.500,NaN,2019-11-6 12:32:56.500,174.50,176.00,166.50,189.75,78.75,138.25,176.75,203.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df = df.fillna(0)

In [13]:
start_time = time.time()

Me_Data=[]
Ind_Data=[]
for i in range(len(Index)-1):	
    T1 = '%s-%s-%s %s:%s:%s.%s' % (Index.index[i].year, Index.index[i].month, Index.index[i].day,\
                                   Index.index[i].hour,Index.index[i].minute,Index.index[i].second,Index.index[i].microsecond)
    T2 = '%s-%s-%s %s:%s:%s.%s' % (Index.index[i+1].year, Index.index[i+1].month, Index.index[i+1].day,\
                                   Index.index[i+1].hour,Index.index[i+1].minute,Index.index[i+1].second,Index.index[i+1].microsecond)
    Time_diff = parse(T2)-parse(T1)    
    Time_diff = Time_diff.seconds + 10**(-6)*(Time_diff.microseconds)
    Data_cut = df[np.argmax((df.number==Index[i]).tolist()):np.argmax((df.number==Index[i+1]).tolist())]    
	#Data_cut=Data.loc[T1:T2]
    Me_Data.append([])
    Ind_Data.append(Index[i])
    Me_Data[i].append(Time_diff)
    
    for channel_name in P_columns:		
        if channel_name =='Pressure 1':
            Me_Data[i].append(np.sum(Data_cut[channel_name]>50))
            Me_Data[i].append(np.max(np.diff(Data_cut[channel_name])))
            Me_Data[i].append(np.min(np.diff(Data_cut[channel_name])))			
            if np.sum(Data_cut[channel_name]>50)==0:
                Me_Data[i].append(0)
                Me_Data[i].append(0)
            else:
                temp_mean = np.mean(Data_cut[channel_name][Data_cut[channel_name]>50])
                Me_Data[i].append(temp_mean)			
                Me_Data[i].append(Data_cut[channel_name].max()-temp_mean)
        else:
            Me_Data[i].append(Data_cut[channel_name].mean())
            Me_Data[i].append(Data_cut[channel_name].max())
            Me_Data[i].append(Data_cut[channel_name].min())
            
    for channel_name in T_columns:
        Me_Data[i].append( Data_cut[channel_name].argmax()/len(Data_cut[channel_name].tolist()) )
        Me_Data[i].append( Data_cut[channel_name+'_trend'].mean() )
        Me_Data[i].append( Data_cut[channel_name+'_detrend'].mean() )
        Me_Data[i].append( Data_cut[channel_name+'_detrend'].max() )
        Me_Data[i].append( Data_cut[channel_name+'_detrend'].min() )
    
Me_Data=np.array(Me_Data)

print(Me_Data.shape)
print("--- %s seconds ---"%(time.time()-start_time))

(502, 47)
--- 4.026716232299805 seconds ---


In [14]:
Fault_indices = np.array(Fault_number)-1
print(len(Fault_number))
print(Fault_number)

## Data Minded	
Data_Minded = Me_Data
Fault_count = 0
Class_label = []
for i in range(len(Ind_Data)):
	if i in Fault_indices:
		Class_label.append([0,1])
		Fault_count+=1
	else:
		Class_label.append([1,0])
Class_label=np.array(Class_label)	
    
# np.savetxt('x_data2.txt', Data_Minded, delimiter=',')
# np.savetxt('y_data2.txt', Class_label, delimiter=',')
np.save( os.path.join(dataset_dir, "./x_data"), Data_Minded )
np.save( os.path.join(dataset_dir, "./y_data"), Class_label )

print(Data_Minded.shape, Class_label.shape)

15
[1, 2, 3, 4, 5, 6, 194, 195, 234, 294, 335, 336, 370, 467, 492]
(502, 47) (502, 2)


In [15]:
## Sigma Detection
Sigma_Detection = np.abs(Data_Minded)>3
Sigma_Detection = np.sum(Sigma_Detection, axis=1)
Sigma_Fault = Sigma_Detection>0
Sigma_NG_Number = np.sum(Sigma_Fault)
print('Sigma_NG_Number: ',Sigma_NG_Number)

Sigma_NG_Number:  502


In [16]:
OK_Success=0
OK_Fail=0
NG_Success=0
NG_Fail=0

for i in range(len(Class_label)):
	if Class_label[i][0] == 1:	# Product OK Case
		if Sigma_Fault[i]:
			OK_Fail+=1	# OK Case Judgement NG
		elif not Sigma_Fault[i]:
			OK_Success+=1  # OK Case Judgement OK
	elif Class_label[i][0] == 0:	# Product NG Case
		if Sigma_Fault[i]:
			NG_Success+=1 # NG Case Judgement NG
		elif not Sigma_Fault[i]: 
			NG_Fail+=1 # NG Case Judgement OK

print('OK_Success: ',OK_Success)
print('OK_Fail: ',OK_Fail)
print('NG_Success: ',NG_Success)
print('NG_Fail: ',NG_Fail)

OK_Success:  0
OK_Fail:  487
NG_Success:  15
NG_Fail:  0


In [17]:
start_time = time.time()

X_Data_OK = []
Y_Data_OK = []
X_Data_NG = []
Y_Data_NG = []
for i in range(len(Data_Minded)):
    if Class_label[i][0] == 1 :
        X_Data_OK.append(Data_Minded[i])
        Y_Data_OK.append(Class_label[i])
    elif Class_label[i][0] == 0 :
        X_Data_NG.append(Data_Minded[i])
        Y_Data_NG.append(Class_label[i])        
X_Data_OK = np.array(X_Data_OK)
Y_Data_OK = np.array(Y_Data_OK)
X_Data_NG = np.array(X_Data_NG)
Y_Data_NG = np.array(Y_Data_NG)

np.save( os.path.join(dataset_dir, "./x_data_OK"), X_Data_OK )
np.save( os.path.join(dataset_dir, "./y_data_OK"), Y_Data_OK )
np.save( os.path.join(dataset_dir, "./x_data_NG"), X_Data_NG )
np.save( os.path.join(dataset_dir, "./y_data_NG"), Y_Data_NG )

print(X_Data_OK.shape, Y_Data_OK.shape)
print(X_Data_NG.shape, Y_Data_NG.shape)
print("--- %s seconds ---"%(time.time()-start_time))

(487, 47) (487, 2)
(15, 47) (15, 2)
--- 0.009010553359985352 seconds ---


In [18]:
# from sklearn.model_selection import train_test_split

# start_time = time.time()

# x_train_OK, x_val_test_OK, y_train_OK, y_val_test_OK = train_test_split(X_Data_OK, Y_Data_OK, test_size=0.398, shuffle=False)
# x_train_NG, x_val_test_NG, y_train_NG, y_val_test_NG = train_test_split(X_Data_NG, Y_Data_NG, test_size=0.4, shuffle=False)
# x_val_OK, x_test_OK, y_val_OK, y_test_OK = train_test_split(x_val_test_OK, y_val_test_OK, test_size=0.5, shuffle=False)
# x_val_NG, x_test_NG, y_val_NG, y_test_NG = train_test_split(x_val_test_NG, y_val_test_NG, test_size=0.5, shuffle=False)

# x_train = np.concatenate([x_train_OK, x_train_NG], axis=0)
# y_train = np.concatenate([y_train_OK, y_train_NG], axis=0)

# x_val = np.concatenate([x_val_OK, x_val_NG], axis=0)
# y_val = np.concatenate([y_val_OK, y_val_NG], axis=0)

# x_test = np.concatenate([x_test_OK, x_test_NG], axis=0)
# y_test = np.concatenate([y_test_OK, y_test_NG], axis=0)

# np.save( os.path.join(dataset_dir, "./x_train_OK"), x_train )
# np.save( os.path.join(dataset_dir, "./y_train_OK"), y_train )
# np.save( os.path.join(dataset_dir, "./x_val_NG"), x_val )
# np.save( os.path.join(dataset_dir, "./y_val_NG"), y_val )
# np.save( os.path.join(dataset_dir, "./x_test_NG"), x_test )
# np.save( os.path.join(dataset_dir, "./y_test_NG"), y_test )

# print(x_train_OK.shape, x_val_OK.shape, x_test_OK.shape)
# print(x_train_NG.shape, x_val_NG.shape, x_test_NG.shape)
# print(x_train.shape, x_val.shape, x_test.shape)

# print("--- %s seconds ---"%(time.time()-start_time))